In [1]:
import pymongo
import dns
import pandas as pd
import io
import datetime

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client.traffic

In [3]:
def inputGantry(gantryID, begin, timeRange):
    #begin=datetime.datetime.strptime(timeStamp, "%Y/%m/%d %H:%M") #"%Y-%m-%d %H:%M") # Time Format2019-06-24 06:40
    end=begin+ datetime.timedelta(minutes = timeRange) #timeRange in minutes
    coll = db['TDCS-M03A']
    match={"GantryId":gantryID, "TimeInterval": {"$gte": begin, "$lt": end}}
    groupby = 'VehicleType'
    group = {'_id': "$%s" % (groupby if groupby else None),
             'total': {'$sum': '$交通量'}}   #將各車型分開加總
    result = coll.aggregate([
            { "$match": match },
            { "$group": group}])
    
    df =  pd.DataFrame(list(result))
    return df

In [5]:
def offInterchange(gantryID, begin, timeRange):
    #begin=datetime.datetime.strptime(timeStamp, "%Y/%m/%d %H:%M") #"%Y-%m-%d %H:%M") # Time Format2019-06-24 06:40
    end=begin+ datetime.timedelta(minutes = timeRange) #timeRange in minutes
    coll = db['TDCS-M08A']
    match={"GantryTo":gantryID, "TimeInterval": {"$gte": begin, "$lt": end}}
    groupby = 'VehicleType'
    group = {'_id': "$%s" % (groupby if groupby else None),
             'total': {'$sum': '$交通量'}}
    result = coll.aggregate([
            { "$match": match },
            { "$group": group}])
    
    df =  pd.DataFrame(list(result))
    return df

In [6]:
def goInterchange(gantryFrom, gantryTo , timeStamp, timeRange):
    fromGntry=inputGantry(gantryFrom, timeStamp, timeRange)
    fromGntry.total = fromGntry.total.apply(lambda x:-x) # to transfer add  to minus
    #print('fromGntry',fromGntry)
    toGntry=inputGantry(gantryTo , timeStamp, timeRange)
    #print('toGntry', toGntry)
    diff=toGntry.set_index('_id').add(fromGntry.set_index('_id'), fill_value=0).reset_index()
    #print('diff',diff)
    
    offIntr=offInterchange(gantryFrom,timeStamp,timeRange)
    #print('offIntr',offIntr)
    result=diff.set_index('_id').add(offIntr.set_index('_id'), fill_value=0).reset_index()
    result.sort_values('_id', inplace=True)
    result.rename(columns={'_id':'VehicleType'}, inplace=True)
    print(result)
    return result

In [7]:
def compareInDiff(timeStamp, timeRange):
    begin=datetime.datetime.strptime(timeStamp, "%Y/%m/%d %H:%M") #"%Y-%m-%d %H:%M")
    print(timeStamp, 'CMS After,')
    dfA=goInterchange('01F2156N', '01F2089N' , begin, timeRange)
    print('CMS Before')
    delta=datetime.timedelta(minutes = timeRange)
    dfB=goInterchange('01F2156N', '01F2089N' , begin-delta, timeRange)

In [8]:
print('The CMS infoed: 204~198K壅塞車速40以下請改道','\n')
compareInDiff("2019/06/25 08:23",60)
print('\n','\n','The CMS NO info','\n')
compareInDiff("2019/06/26 08:23",60)
compareInDiff("2019/06/27 08:23",60)
compareInDiff("2019/06/28 08:23",60)
compareInDiff("2019/06/24 08:23",60)

The CMS infoed: 204~198K壅塞車速40以下請改道 

2019/06/25 08:23 CMS After,
   VehicleType  total
0            5     11
1           31    434
2           32    245
3           41      4
4           42     59
CMS Before
   VehicleType  total
0            5     32
1           31    867
2           32    323
3           41      3
4           42     52

 
 The CMS NO info 

2019/06/26 08:23 CMS After,
   VehicleType  total
0            5     21
1           31    432
2           32    229
3           41      6
4           42     71
CMS Before
   VehicleType  total
0            5     23
1           31    773
2           32    274
3           41     -5
4           42     41
2019/06/27 08:23 CMS After,
   VehicleType  total
0            5     29
1           31    402
2           32    217
3           41      3
4           42     60
CMS Before
   VehicleType  total
0            5   20.0
1           31  756.0
2           32  261.0
3           41    9.0
4           42   51.0
2019/06/28 08:23 CMS After,
   